In [ ]:
epsg = 3763

origin = (0, 0)

dist = 500

angle_min = 0
angle_max = 45

angles_int = 45

inshp  = '/mnt/d/a2land2023/rule3_4/sample.shp'
outshp = '/mnt/d/a2land2023/rule3_4/buffer_res.shp'

In [ ]:
import numpy as np
import geopandas as gp
from glass.gobj import create_polygon
from glass.rd.shp import shp_to_obj
from glass.wt.shp import df_to_shp
from glass.pd.dagg import col_listwlist_to_row

import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [ ]:
def run_buffer(r):
    #nvtx = len(r.geometry.exterior.coords)
    
    #side_len = r.perimeter / nvtx
    multipoly = []
    
    minangle = 0
    maxangle = angles_int
    
    while maxangle <= 360:
        coords = [(r.x, r.y)]
        for i in range(minangle, maxangle + 1):
            y = r.y + ((r.dist + dist) * np.cos(np.radians(i)))
            x = r.x + ((r.dist + dist) * np.sin(np.radians(i)))
    
            coords.append((x, y))
        
        coords.append((r.x, r.y))   
    
        poly = create_polygon(coords, api='shapely')
        
        multipoly.append([r.featid, poly, f"{str(minangle)} - {str(maxangle)}"])
        
        minangle += angles_int
        maxangle += angles_int
    
    r["geoms"] = multipoly
    
    return r

In [ ]:
idf = shp_to_obj(inshp)

In [ ]:
s = idf.geometry

t = gp.GeoSeries(gp.points_from_xy(s.envelope.bounds.maxx, s.envelope.bounds.maxy))
idf["dist"] = s.envelope.centroid.distance(t)

idf['x'] = s.envelope.centroid.x
idf['y'] = s.envelope.centroid.y

idf["featid"] = idf.index

idf = idf.apply(lambda x: run_buffer(x), axis=1)

idf.drop(["x", "y", "geometry"], axis=1, inplace=True)

In [ ]:
idf = col_listwlist_to_row(
    idf, "geoms", ["ofid", "geom", "direction"],
    geomcol="geom", epsg=epsg
)

df_to_shp(idf, outshp)

In [ ]:
idf

In [ ]:
#generate geoseries of maxx and maxy of each polygon's envelope
s = idf.geometry

t = gp.GeoSeries(gp.points_from_xy(s.envelope.bounds.maxx, s.envelope.bounds.maxy))

fig, ax = plt.subplots()
#circle with radius of distance from centroid of envelope to it's maxx, maxy 
s.envelope.centroid.buffer(s.envelope.centroid.distance(t)).plot(ax=ax, color='green', alpha=0.5)
#envelope of the polygon
s.envelope.plot(ax=ax, color='magenta', alpha=0.4)
#polygon
s.plot(ax=ax, color='red', alpha=0.4)
#points of envelope's maxx, maxy
t.plot(ax=ax, color='blue')
plt.show()